In [3]:
import socket
import sticks

game_socket = socket.socket()
game_socket.connect(('127.0.0.1', 6789))

recieved = b''
recieved += game_socket.recv(1024)

game = sticks.sticks(int(recieved.decode()))
while True:
    game.render_sticks()
    
    recieved=b''
    opp_move = ''
    print('Waiting for Opponent...')
    while b'\n' not in recieved:
        recieved += game_socket.recv(1024)
    if recieved.startswith(b'WON'):
        opp_move = recieved.decode()   
        opp_move = opp_move[3:]
        col = str(opp_move[0])
        row = str(opp_move[1])
        amt = str(opp_move[2])
        game.remove_sticks(col,row,amt) 
        game.render_sticks()
        print("Opponent Won!")
        break
    if recieved.startswith(b'MOVE'):
        opp_move = recieved.decode()   
        opp_move = opp_move[4:]
        
    col = str(opp_move[0])
    row = str(opp_move[1])
    amt = str(opp_move[2])
    game.remove_sticks(col,row,amt) 
    game.render_sticks()
    
    # get input from player
    col = input("Select Column: ")
    row = input("Select Row: ")
    amt = input("Select Amount: ")
           
    while game.remove_sticks(col, row, amt) == True:
        print("Your input is invalid. Please try again.")
        col = input("Select Column: ")
        row = input("Select Row: ")
        amt = input("Select Amount: ")
        
    game.remove_sticks(col, row, amt)
    
    # if after player turn. left 1 stick, they win
    if game.sticks == 1:
        game_socket.sendall(b'WON' + str(col).encode() + str(row).encode() + str(amt).encode()+ b'\n')
        game.render_sticks()
        print("You Won!")
        break  
        
    game_socket.sendall(b'MOVE' + str(col).encode() + str(row).encode() + str(amt).encode()+ b'\n')
        

game_socket.close()

Remaining Sticks: 6
  0 1 2
0 1
1 1 1
2 1 1 1
Waiting for Opponent...
Remaining Sticks: 3
  0 1 2
0 1
1 1 1
2 - - -
Select Column: 0
Select Row: 0
Select Amount: 1
Remaining Sticks: 2
  0 1 2
0 -
1 1 1
2 - - -
Waiting for Opponent...
Remaining Sticks: 1
  0 1 2
0 -
1 1 -
2 - - -
Opponent Won!
